In [199]:
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
import codecs
import pandas as pd
import time

hexlify = codecs.getencoder('hex')

def first_word(kanji):
  temp = kanji.encode("utf-8")
  temp = hexlify(temp)[0].decode("utf-8").upper()
  q = ""
  for i, h in enumerate(temp):
    if i % 2 == 0:
      q += "%"
    
    q += h

  url = f"https://jisho.org/search/{q}"
  header = {'User-Agent': 'Mozilla/5.0'}
  req = Request(url, headers=header)

  html = urlopen(req)
  soup = BeautifulSoup(html)

  div = soup.find("div", {"class": "concept_light-representation"})
  text = div.find("span", {"class": "text"}).text

  return text.replace("\n", "").replace(" ", "")

def getResults(kanji):
  temp = kanji.encode("utf-8")
  temp = hexlify(temp)[0].decode("utf-8").upper()
  q = ""
  for i, h in enumerate(temp):
    if i % 2 == 0:
      q += "%"
    
    q += h

  url = f"https://www.google.co.jp/search?q={q}&tbm=isch"
  header = {'User-Agent': 'Mozilla/5.0'}
  req = Request(url, headers=header)

  html = urlopen(req)
  soup = BeautifulSoup(html)

  imgs = soup.find_all("img")
  temp = []
  for img in imgs:
    temp.append(img["src"].replace("https://encrypted-tbn0.gstatic.com/images?q=tbn:", ""))

  return temp[1:9]

In [200]:
df = pd.read_csv("/content/core_kanji.csv")
all_kanji = df["New (Shinjitai)"]

In [203]:
urls = []
for kanji in all_kanji:
  # kanji = first_word(kanji)
  urls.append(getResults(kanji))

time.sleep(3)

df["URLs"] = urls

In [204]:
urls
# https://encrypted-tbn0.gstatic.com/images?q=tbn:


[['ANd9GcQpTj_u-Ldv3LV-PGoVFjbPA5vJkrJZd7DeQLEIHTCx0TWNl7RfUdA1b7l3Nic&s',
  'ANd9GcRX4x0y_SpTc8-w7AEwANtmnfbKrBjCiWH03tMHsD0U4CAV9FYCMwECF4qaGg&s',
  'ANd9GcRUlaOsiw1V9Np_jfKDjxrx9MbcqQDhvJ8WQegpzm_BT84e0J7FZMeYgQ-tBZw&s',
  'ANd9GcTOWWAsyE9wN4TxoN5VoJZGyjSxWu6bpmcu622wxROnxWL5d0ExT4F3jaVhfvs&s',
  'ANd9GcSHMZ4KEAA77Tje0zNlgUB-0XYQIFrE9qaKJdO8Kc0a3DAMS7hcfOerh3ZExw&s',
  'ANd9GcSpKZM4jCpBZi8ILJqrvQbLqqGajuh9DkyHsFUAJEmvLdrPPE-kzMG6fb9MwKw&s',
  'ANd9GcR04Onuk3Gi-u1GTzceXqX2nshowzJ3j4caXk-NWNNS0pwnzQolkxYMG4d4IA&s',
  'ANd9GcSvBSS8a3s8zIO-9c39j5PJlp5TDm5tyHrgqnf1G8B0x9c-NI52PH0KFTO4brM&s'],
 ['ANd9GcRJG7mL4qOM7MlwL-LdJqlGGwl1sERGQzPABc3hDYorKLH_idXvdhSGoQVbzII&s',
  'ANd9GcQN9DEz8SL1EqPPN3cKH_NgKWz5x23xlRzMuf9FOgZXKhhMkhKNlHsnciuRzw&s',
  'ANd9GcTsiEWm0TSHtvKRYTXcMyGHEzpW7SOOrfpUnKoyvbpEOkzYCKpqcWzoaMn6uSQ&s',
  'ANd9GcS_LL2YdY0A8xWIroInForhwhGv1Io0y_RJrf_acLdc_w39x8QuuaQeqv8RuA&s',
  'ANd9GcTRbP0k3loH1CWHRJSQi6AIQf3Rnzidhh-Bkz_MtIKyG-fqxaqJuCFuvj42YA&s',
  'ANd9GcTRUGeUTzA98uHzEqfc1KN

In [195]:
def parseReadings(readings):
  hiragana = {'こ', 'て', 'づ', 'ぺ', 'え', 'か', 'き', 'ね', 'い', 'み', 'ぎ', 'ば', 'ふ', 'ぢ', 'ご', 'ぶ', 'も', 'り', 'お', 'れ', 'げ', 'と', 'さ', 'ん', 'う', 'ち', 'へ', 'は', 'せ', 'し', 'な', 'す', 'ず', 'ぷ', 'ろ', 'た', 'ほ', 'よ', 'ら', 'そ', 'が', 'や', 'ど', 'け', 'だ', 'く', 'つ', 'ぴ', 'の', 'を', 'ぐ', 'わ', 'べ', 'に', 'で', 'ぬ', 'ま', 'む', 'ゆ', 'ぞ', 'あ', 'じ', 'び', 'ぱ', 'ぽ', 'め', 'ひ', 'ぼ', 'ざ', 'る', 'ぜ'}
  splits = ["\n", "\\", "\a", "\b", "\f", "\r", "\t", "\v", "\o"]

  temp2 = readings.split("、")
  for t in temp2:
    for s in splits:
      if len(t.split(s)) > 1 or t.split(s)[0][0] in hiragana:
        return t.split(s)[0]
  return temp2[0]

temp = []
for reading in df["Readings"]:
  temp.append(parseReadings(reading))

df["Readings"] = temp

80

In [212]:
temp = []
for reading in df["Readings"]:
  temp.append(parseReadings(reading))

df["Readings"] = temp
df = df.drop(columns=["#", "Old (Kyūjitai)", "Strokes", "Year added", "Grade"])

In [213]:
df.to_csv("/content/kanji_with_image_link.csv")